In [91]:
import pandas as pd
import numpy as np
import altair as alt
import calendar
from IPython.display import display
from google.colab import drive



#Import Data



In [92]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
#นำเข้า dataset 10 series & movies with most hours streamed 2021-2024
most_hour_path = '/content/drive/MyDrive/20241227globaldata.csv'
df_most_hour = pd.read_csv(most_hour_path, encoding="utf-8", delimiter=",")

#เนื่องจาก dataset แรก ไม่ระบุ genre จึงนำเข้าข้อมูลที่มี genre เพิ่มเติม
series_2023_path = '/content/drive/MyDrive/Shows2023.csv'
df_series_2023 = pd.read_csv(series_2023_path, encoding="utf-8", delimiter=",")
sereis_2022_2024_path = '/content/drive/MyDrive/Best_Shows_Netflix_2022-2024.csv'
df_sereis_2022_2024 = pd.read_csv(sereis_2022_2024_path, encoding="utf-8", delimiter=",")

df_most_hour.head()

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,runtime_override_flag,episode_launch_dtls
0,2024-12-22,Films (English),1,Carry-On,NaN,110200000,2.0000,55100000.0,2,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
1,2024-12-22,Films (English),2,The Six Triple Eight,NaN,30900000,2.1667,14300000.0,1,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
2,2024-12-22,Films (English),3,Dr. Seuss' The Grinch,NaN,16300000,1.4333,11400000.0,10,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
3,2024-12-22,Films (English),4,That Christmas,NaN,17800000,1.6000,11100000.0,3,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
4,2024-12-22,Films (English),5,Disaster Holiday,NaN,11600000,1.5500,7500000.0,2,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN


# Preprocessing Data

In [94]:
#ตัด English กับ Non-english ออกจากคอลัมน์ category เนื่องจากตั้งการวิเคราะห์ทั้งสองประเภทพร้อมกันเพื่อหาเรื่องที่มียอดวิวสูงสุด
df_most_hour['category'] = df_most_hour['category'].str.extract(r'^(.*?)\s*\(')[0]
df_most_hour_interrest = df_most_hour[['week', 'category', 'show_title', 'weekly_hours_viewed']]
df_most_hour_interrest = df_most_hour_interrest.sort_values(by='week').reset_index(drop=True)

In [95]:
# เช็กชื่อคอลัมน์ที่ต้องการนำมา merg
print(df_most_hour_interrest.columns)
print(df_series_2023.columns)
print(df_sereis_2022_2024.columns)

Index(['week', 'category', 'show_title', 'weekly_hours_viewed'], dtype='object')
Index(['Unnamed: 0', 'id', 'name', 'year', 'certificate', 'runtime', 'genre',
       'ratings'],
      dtype='object')
Index(['index', 'TITLE', 'RELEASE_YEAR', 'SCORE', 'NUMBER_OF_VOTES',
       'DURATION', 'NUMBER_OF_SEASONS', 'MAIN_GENRE', 'MAIN_PRODUCTION'],
      dtype='object')


In [96]:
# เปลี่ยนชื่อคอลัมน์ที่ต้องการนำมา merg
df_series_2023 = df_series_2023.rename(columns= {'name': 'show_title_2023', 'genre': 'genre_2023'})
df_sereis_2022_2024 = df_sereis_2022_2024.rename(columns= {'TITLE': 'show_title_2022', 'MAIN_GENRE': 'genre_2022'})
print(df_series_2023.columns)
print(df_sereis_2022_2024.columns)

Index(['Unnamed: 0', 'id', 'show_title_2023', 'year', 'certificate', 'runtime',
       'genre_2023', 'ratings'],
      dtype='object')
Index(['index', 'show_title_2022', 'RELEASE_YEAR', 'SCORE', 'NUMBER_OF_VOTES',
       'DURATION', 'NUMBER_OF_SEASONS', 'genre_2022', 'MAIN_PRODUCTION'],
      dtype='object')


In [97]:
#เลือกแต่ข้อมูล TV
df_most_hour_eng = df_most_hour_interrest[df_most_hour_interrest['category'] == 'TV'].copy()
df_most_hour_eng = df_most_hour_eng.reset_index(drop=True)

ทำการรวมยอดวิวในแต่ละสัปดาห์เพื่อหา 10 อันดับซีรีส์ที่มียอดวิวสูงสุดในแต่ละเดือน

In [98]:
# แปลงข้อมูลในคอลัมน์ week เป็น datetime
df_most_hour_eng['week'] = pd.to_datetime(df_most_hour_eng['week'])

#สร้างคอมลัมน์ year กับ month
df_most_hour_eng['year'] = df_most_hour_eng['week'].dt.year
df_most_hour_eng['month'] = df_most_hour_eng['week'].dt.month

# grouping ข้อมูล โดยแบ่งข้อมูลตาม year, month, และ show_title เพื่อคำนวณยอดวิวรวม
df_most_hour_eng = df_most_hour_eng.groupby(['year', 'month', 'show_title'], as_index=False)['weekly_hours_viewed'].sum()

#หาสิบอันดับซีรีส์ที่มียอดวิวสูงสุดในแต่ละเดือน
df_most_hour_eng = df_most_hour_eng.sort_values(['year', 'month', 'weekly_hours_viewed'], ascending=[True, True, False]) \
                                    .groupby(['year', 'month']) \
                                    .head(10) \
                                    .reset_index(drop=True)

df_most_hour_eng = df_most_hour_eng.rename(columns={'weekly_hours_viewed': 'monthly_hours_viewed'})

# แสดงผล
df_most_hour_eng


,year,month,show_title,monthly_hours_viewed
0,2021,7,Virgin River,273540000
1,2021,7,Sex/Life,232790000
2,2021,7,Manifest,147240000
3,2021,7,Never Have I Ever,125970000
4,2021,7,Elite,115020000
...,...,...,...,...
415,2024,12,Senna,83200000
416,2024,12,Virgin River,67700000
417,2024,12,The Trunk,66700000
418,2024,12,Arcane,61900000


เติมข้อมูล genre โดยการ merge กับ dataset อื่น เนื่องจากซีรีส์แต่ละเรื่องมีหลาย genre รวมกัน จึงทำการแยกของมาแล้วใส่ในคอลัมน์แยก ซีรีส์เรื่องไหนที่มี genre ไม่ถึงสาม genre จะทำการ copy genre เดินใส่ลงไปเพื่อให้ค่าไม่เป็น Nan

In [99]:
# ขั้นตอนที่ 1: ทำให้ key ใน DataFrame ที่ merge ด้วยเป็นแบบ unique
df_series_2023_unique = df_series_2023[['show_title_2023', 'genre_2023']].drop_duplicates(subset='show_title_2023')
df_series_2022_unique = df_sereis_2022_2024[['show_title_2022', 'genre_2022']].drop_duplicates(subset='show_title_2022')

# ขั้นตอนที่ 2: Merge กับ df_series_2023
df_most_hour_eng = df_most_hour_eng.merge(
    df_series_2023_unique,
    left_on='show_title',
    right_on='show_title_2023',
    how='left',
    indicator=True
)

# กำหนดค่า genre จาก genre_2023 ที่ merge เข้ามา
df_most_hour_eng['genre'] = df_most_hour_eng['genre_2023']

# ขั้นตอนที่ 3: Merge กับ df_series_2022 เฉพาะแถวที่ยังไม่มีค่า genre
df_most_hour_eng = df_most_hour_eng.merge(
    df_series_2022_unique,
    left_on='show_title',
    right_on='show_title_2022',
    how='left'
)

# เติม genre จาก genre_2022 เฉพาะแถวที่ยังไม่มีค่า
df_most_hour_eng.loc[
    df_most_hour_eng['genre'].isna(), 'genre'
] = df_most_hour_eng['genre_2022']

# ขั้นตอนที่ 4: ลบคอลัมน์ที่ไม่ต้องการ
df_most_hour_eng = df_most_hour_eng.drop(
    columns=['show_title_2023', 'genre_2023', 'show_title_2022', 'genre_2022', '_merge']
)

# ขั้นตอนที่ 5: ลบ '\n' ที่เกินมา และ trim space
df_most_hour_eng['genre'] = df_most_hour_eng['genre'].str.strip()

# ขั้นตอนที่ 6: แยก genre เป็นคอลัมน์ใหม่
# แยกค่าในคอลัมน์ genre ออกเป็นหลายคอลัมน์โดยใช้ `,` เป็นตัวแบ่ง
genre_split = df_most_hour_eng['genre'].str.split(',', expand=True)

# เปลี่ยนชื่อคอลัมน์ใหม่ให้เป็น Genre_1, Genre_2 และ Genre_3
genre_split.columns = [f'Genre_{i+1}' for i in range(genre_split.shape[1])]

# รวมกลับเข้า df_most_hour_eng
df_most_hour_eng = pd.concat([df_most_hour_eng, genre_split], axis=1)

# แสดงผลลัพธ์ของ DataFrame ที่อัปเดตแล้ว
df_most_hour_eng

,year,month,show_title,monthly_hours_viewed,genre,Genre_1,Genre_2,Genre_3
0,2021,7,Virgin River,273540000,"Drama, Romance",Drama,Romance,None
1,2021,7,Sex/Life,232790000,"Comedy, Drama, Romance",Comedy,Drama,Romance
2,2021,7,Manifest,147240000,"Drama, Mystery, Sci-Fi",Drama,Mystery,Sci-Fi
3,2021,7,Never Have I Ever,125970000,"Comedy, Drama",Comedy,Drama,None
4,2021,7,Elite,115020000,"Crime, Drama, Thriller",Crime,Drama,Thriller
...,...,...,...,...,...,...,...,...
415,2024,12,Senna,83200000,drama,drama,None,None
416,2024,12,Virgin River,67700000,"Drama, Romance",Drama,Romance,None
417,2024,12,The Trunk,66700000,"romance, mystery",romance,mystery,None
418,2024,12,Arcane,61900000,"Animation, Action, Adventure",Animation,Action,Adventure


In [100]:
# ตรวจสอบว่ามีแถวที่มีค่า NaN ในคอลัมน์ 'genre' หรือไม่
nan_genre = df_most_hour_eng[df_most_hour_eng['genre'].isna()]
print("แถวที่มีค่า NaN ในคอลัมน์ 'genre':")
print(nan_genre)

แถวที่มีค่า NaN ในคอลัมน์ 'genre':
Empty DataFrame
Columns: [year, month, show_title, monthly_hours_viewed, genre, Genre_1, Genre_2, Genre_3]
Index: []


In [101]:
#หาว่าใน column genre มี value อะไรบ้างเพื่อทำการเปลี่ยนให้มีรูปแบบการสะกดเหมือนกัน
unique_genres = df_most_hour_eng['genre'].unique()
print(unique_genres)

['Drama, Romance' 'Comedy, Drama, Romance' 'Drama, Mystery, Sci-Fi'
 'Comedy, Drama' 'Crime, Drama, Thriller' 'Game-Show, Reality-TV, Romance'
 'comedy, drama' 'Action, Crime, Drama' 'Drama, Sport' 'Crime, Drama'
 'Drama' 'Drama, Thriller' 'Adventure, Comedy, Drama'
 'Action, Drama, Mystery' 'Crime, Drama, Fantasy'
 'thriller, crime, mystery' 'Comedy, Crime, Drama' 'documentary'
 'Comedy, Romance' 'Crime, Drama, Romance' 'drama' 'thriller'
 'Drama, Fantasy, Horror' 'action' 'Crime, Drama, Mystery' 'Drama, Music'
 'Animation, Action, Adventure' 'Action, Adventure, Drama'
 'Adventure, Drama, Family' 'Action, Adventure, Crime' 'drama, fantasy'
 'Action, Comedy, Drama' 'crime, thriller, comedy'
 'Drama, Horror, Mystery' 'Action, Drama, Fantasy'
 'drama, thriller, crime' 'Drama, Mystery, Thriller' 'Drama, Sci-Fi'
 'Reality-TV, Romance' 'comedy, crime, thriller' 'Drama, History, Romance'
 'thriller, mystery, crime' 'Action, Drama, History'
 'crime, documentary, thriller' 'Reality-TV' 'Action

In [102]:
# ดรอปคอลัมน์ 'genre'
df_most_hour_eng = df_most_hour_eng.drop(columns=['genre'])

# แทนค่า None หรือ NaN ใน Genre_2 และ Genre_3 ด้วยค่าใน Genre_1
df_most_hour_eng['Genre_2'] = df_most_hour_eng['Genre_2'].fillna(df_most_hour_eng['Genre_1'])
df_most_hour_eng['Genre_3'] = df_most_hour_eng['Genre_3'].fillna(df_most_hour_eng['Genre_1'])

# แสดงผลลัพธ์
df_most_hour_eng

,year,month,show_title,monthly_hours_viewed,Genre_1,Genre_2,Genre_3
0,2021,7,Virgin River,273540000,Drama,Romance,Drama
1,2021,7,Sex/Life,232790000,Comedy,Drama,Romance
2,2021,7,Manifest,147240000,Drama,Mystery,Sci-Fi
3,2021,7,Never Have I Ever,125970000,Comedy,Drama,Comedy
4,2021,7,Elite,115020000,Crime,Drama,Thriller
...,...,...,...,...,...,...,...
415,2024,12,Senna,83200000,drama,drama,drama
416,2024,12,Virgin River,67700000,Drama,Romance,Drama
417,2024,12,The Trunk,66700000,romance,mystery,romance
418,2024,12,Arcane,61900000,Animation,Action,Adventure


In [103]:
# กำหนดคอลัมน์ genre ที่ต้องการปรับเปลี่ยน
genre_cols = ['Genre_1', 'Genre_2', 'Genre_3']

# แปลงค่าทุกคอลัมน์ให้ตัวแรกเป็นพิมพ์ใหญ่ ตัวอื่นพิมพ์เล็ก และลบช่องว่าง
for col in genre_cols:
    df_most_hour_eng[col] = df_most_hour_eng[col].astype(str).apply(lambda x: x.strip().capitalize())

genre_mapping = {
    'Reality-tv': 'Reality',
    'Scifi': 'Sci-fi',
    'Advanture': 'Adventure'
}

# ใช้ .replace() กับทุกคอลัมน์
for col in genre_cols:
    df_most_hour_eng[col] = df_most_hour_eng[col].replace(genre_mapping)

# ดึงค่า unique จากทุกคอลัมน์แล้วรวมเป็น list เดียว
unique_genres = sorted(pd.unique(df_most_hour_eng[genre_cols].values.ravel()))

# แสดงผลลัพธ์
print(unique_genres)


['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantacy', 'Fantasy', 'Game-show', 'History', 'Horror', 'Music', 'Mystery', 'Reality', 'Romance', 'Sci-fi', 'Sport', 'Thriller', 'War', 'Western']


In [104]:
df_most_hour_eng

,year,month,show_title,monthly_hours_viewed,Genre_1,Genre_2,Genre_3
0,2021,7,Virgin River,273540000,Drama,Romance,Drama
1,2021,7,Sex/Life,232790000,Comedy,Drama,Romance
2,2021,7,Manifest,147240000,Drama,Mystery,Sci-fi
3,2021,7,Never Have I Ever,125970000,Comedy,Drama,Comedy
4,2021,7,Elite,115020000,Crime,Drama,Thriller
...,...,...,...,...,...,...,...
415,2024,12,Senna,83200000,Drama,Drama,Drama
416,2024,12,Virgin River,67700000,Drama,Romance,Drama
417,2024,12,The Trunk,66700000,Romance,Mystery,Romance
418,2024,12,Arcane,61900000,Animation,Action,Adventure


#10 อันดับซีรีส์ยอดวิวสูงสุดในแต่ละปี

In [105]:
#รวมยอดวิวของแต่ละเรื่องแบ่งเป็นปีแล้วจัดอันดับ 10 อันดับแรกของแต่ละปี
grouped_data = df_most_hour_eng.groupby(['year', 'show_title'])['monthly_hours_viewed'].sum().reset_index()
sorted_data = grouped_data.sort_values(['year', 'monthly_hours_viewed'], ascending=[True, False])
top_10_shows_per_year = {}

for year in sorted_data['year'].unique():
    top_10_shows_per_year[year] = sorted_data[sorted_data['year'] == year].head(10)

for year, df in top_10_shows_per_year.items():
    print(f"Top 10 Shows in {year}:")
    print(df)
    print("-" * 20)


Top 10 Shows in 2021:
    year         show_title  monthly_hours_viewed
30  2021         Squid Game            2226950000
22  2021        Money Heist            1060200000
43  2021                You             764990000
19  2021               Maid             655510000
28  2021      Sex Education             561360000
27  2021        Outer Banks             445550000
38  2021        The Witcher             434920000
36  2021  The Queen of Flow             364190000
42  2021       Virgin River             333920000
17  2021            Lucifer             269310000
--------------------
Top 10 Shows in 2022:
     year                  show_title  monthly_hours_viewed
95   2022             Stranger Things            2897900000
120  2022                   Wednesday            1314690000
53   2022                  Bridgerton             939020000
57   2022                      DAHMER             934220000
81   2022                    Manifest             849040000
86   2022                

In [106]:
# ✅ รวมยอดวิวของแต่ละซีรีส์ตลอดทั้งปี
df_yearly = df_most_hour_eng.groupby(['year', 'show_title'], as_index=False)['monthly_hours_viewed'].sum()

# ✅ แปลงเป็นหน่วย "พันชั่วโมง"
df_yearly['yearly_hours_viewed_k'] = df_yearly['monthly_hours_viewed'] / 1000

# ✅ กำหนดสเกลสูงสุดเป็น 3,000,000 พันชั่วโมง (3 พันล้านชั่วโมง)
max_x_scale = 3_000_000

# ✅ ฟังก์ชันสำหรับเลือก 10 อันดับแรกของแต่ละปี
def get_top_10(df, year):
    return df[df['year'] == year].nlargest(10, 'monthly_hours_viewed')

# ✅ ดึงข้อมูลของแต่ละปี
df_2021 = get_top_10(df_yearly, 2021)
df_2022 = get_top_10(df_yearly, 2022)
df_2023 = get_top_10(df_yearly, 2023)
df_2024 = get_top_10(df_yearly, 2024)

# ✅ ฟังก์ชันสร้างกราฟ
def plot_top_10_chart(df, year):
    return alt.Chart(df).mark_bar().encode(
        x=alt.X('yearly_hours_viewed_k:Q', title='Total Yearly Hours Viewed (Thousands of Hours)',
                scale=alt.Scale(domain=[0, max_x_scale])),
        y=alt.Y('show_title:N', title='Show Title', sort='-x'),
        tooltip=['show_title', 'yearly_hours_viewed_k']
            ).properties(title=f'Top 10 Shows in {year}')


# ✅ สร้างกราฟของแต่ละปี
top_10_chart_2021 = plot_top_10_chart(df_2021, 2021)
top_10_chart_2022 = plot_top_10_chart(df_2022, 2022)
top_10_chart_2023 = plot_top_10_chart(df_2023, 2023)
top_10_chart_2024 = plot_top_10_chart(df_2024, 2024)

# ✅ แสดงกราฟทั้งหมด
(top_10_chart_2021 | top_10_chart_2022) & (top_10_chart_2023 | top_10_chart_2024)

alt.VConcatChart(...)

In [107]:
df_2021

,year,show_title,monthly_hours_viewed,yearly_hours_viewed_k
30,2021,Squid Game,2226950000,2226950.0
22,2021,Money Heist,1060200000,1060200.0
43,2021,You,764990000,764990.0
19,2021,Maid,655510000,655510.0
28,2021,Sex Education,561360000,561360.0
27,2021,Outer Banks,445550000,445550.0
38,2021,The Witcher,434920000,434920.0
36,2021,The Queen of Flow,364190000,364190.0
42,2021,Virgin River,333920000,333920.0
17,2021,Lucifer,269310000,269310.0


In [108]:
df_2022

,year,show_title,monthly_hours_viewed,yearly_hours_viewed_k
95,2022,Stranger Things,2897900000,2897900.0
120,2022,Wednesday,1314690000,1314690.0
53,2022,Bridgerton,939020000,939020.0
57,2022,DAHMER,934220000,934220.0
81,2022,Manifest,849040000,849040.0
86,2022,Ozark,725720000,725720.0
55,2022,Café con aroma de mujer,713260000,713260.0
75,2022,Inventing Anna,654560000,654560.0
56,2022,Cobra Kai,624010000,624010.0
65,2022,Extraordinary Attorney Woo,577240000,577240.0


In [109]:
df_2023

,year,show_title,monthly_hours_viewed,yearly_hours_viewed_k
154,2023,Ginny & Georgia,788880000,788880.0
199,2023,The Night Agent,699950000,699950.0
195,2023,The Glory,560990000,560990.0
172,2023,Outer Banks,494640000,494640.0
169,2023,ONE PIECE,480300000,480300.0
216,2023,You,437800000,437800.0
177,2023,Queen Charlotte: A Bridgerton Story,432250000,432250.0
211,2023,Wednesday,414790000,414790.0
156,2023,King the Land,402100000,402100.0
204,2023,The Witcher,384400000,384400.0


In [110]:
df_2024

,year,show_title,monthly_hours_viewed,yearly_hours_viewed_k
234,2024,Bridgerton,1239400000,1239400.0
275,2024,Queen of Tears,653900000,653900.0
246,2024,Fool Me Once,581100000,581100.0
265,2024,Monsters,491800000,491800.0
274,2024,Prison Break,478300000,478300.0
227,2024,Avatar The Last Airbender,442600000,442600.0
291,2024,The Gentlemen,441600000,441600.0
260,2024,Love Is Blind,410500000,410500.0
218,2024,3 Body Problem,334100000,334100.0
249,2024,Griselda,334100000,334100.0


#รวมยอดวิวของประเภทซีรีส์ในแต่ละปี

In [118]:
#Drop ปี 2021 เนื่องจากมีข้อมูลเพียงครึ่งปี จึงไม่สามารถนำยอดวิวมาเปรียบเทียบกับปัอื่นที่รวมยอดวิวของทั้งปีได้
df_most_hour_eng_22_24 = df_most_hour_eng[df_most_hour_eng['year'] != 2021].reset_index(drop=True)

# แปลงข้อมูลให้ Genre อยู่ในรูปแบบที่เหมาะสม
genre_views = df_most_hour_eng_22_24.melt(
    id_vars=['year', 'monthly_hours_viewed'],
    value_vars=['Genre_1', 'Genre_2', 'Genre_3'],
    var_name='Genre_Type',
    value_name='Genre'
).groupby(['year', 'Genre'])['monthly_hours_viewed'].sum().reset_index()

# กราฟแท่งแสดงยอดวิวของแต่ละประเภทหนังในแต่ละปี
spaced_colors = {
    'Action': '#6A5ACD', 'Comedy': '#F4A261', 'Horror': '#264653', 'Sci-fi': '#2A9D8F',
    'Documentary': '#E9C46A', 'Mystery': '#98FB98', 'Reality': '#1D3557', 'Fantasy': '#FF69B4',
    'Crime': '#A8DADC', 'Adventure': '#457B9D', 'Biography': '#D90429', 'Thriller': '#6D597A',
    'Romance': '#43AA8B', 'Drama': '#F94144', 'Family': '#4D908E', 'Animation': '#F3722C'
}

genre_chart = alt.Chart(genre_views).mark_bar().encode(
    x=alt.X('Genre:N', title="Genre", sort='-y'),
    y=alt.Y('monthly_hours_viewed:Q', title="Total Monthly Hours Viewed"),
    color=alt.Color('Genre:N',
                    legend=alt.Legend(title="Genre"),
                    scale=alt.Scale(domain=list(spaced_colors.keys()), range=list(spaced_colors.values()))
                   ),
    column='year:N',
    tooltip=['year', 'Genre', 'monthly_hours_viewed']
).properties(
    width=300,
    height=500,
)

genre_chart

alt.Chart(...)

จากกราฟแท่งด้านบน ต้องการดูข้อมูลของ 5 อันดับซีรีส์ที่มียอยวิวสูงสุดในแต่ละปี เพื่อดูแนวโน้มการเปลี่ยนแปลง

In [117]:
#Drop ปี 2021 เนื่องจากมีข้อมูลเพียงครึ่งปี จึงไม่สามารถนำยอดวิวมาเปรียบเทียบกับปัอื่นที่รวมยอดวิวของทั้งปีได้
df_most_hour_eng_22_24 = df_most_hour_eng[df_most_hour_eng['year'] != 2021].reset_index(drop=True)

# กำหนดสีที่ต้องการให้แต่ละประเภทหนัง
genre_colors = {
    'Drama': '#F94144',
    'Comedy': '#F4A261',
    'Crime': '#A8DADC',
    'Thriller': '#6D597A',
    'Romance': '#43AA8B',
    'Action': '#6A5ACD'
}

# พล็อตกราฟแท่ง พร้อมกำหนดสีที่ต้องการ
top5_genre_chart = alt.Chart(top5_genres_trend).mark_bar().encode(
    x=alt.X('year:O', title="Year"),
    y=alt.Y('monthly_hours_viewed:Q', title="Total Monthly Hours Viewed"),
    color=alt.Color('Genre:N',
                    legend=alt.Legend(title="Genre"),
                    scale=alt.Scale(domain=list(genre_colors.keys()), range=list(genre_colors.values()))  # กำหนดสีให้ตรงกับ Genre
                   ),
    column=alt.Column('Genre:N', title="Genre"),
    tooltip=['year', 'Genre', 'monthly_hours_viewed']
).properties(
    width=150,
    height=400
)

top5_genre_chart


alt.Chart(...)

#Heatmap แสดงการกระจายตัวของ Genre ในแต่ละปี

In [119]:
# แปลงข้อมูลให้อยู่ในรูปแบบ Long Format และลบค่าซ้ำ
df_melted = (
    df_most_hour_eng_22_24.melt(
        id_vars=['year', 'show_title'],  # เก็บคอลัมน์ 'year' และ 'show_title' ไว้
        value_vars=['Genre_1', 'Genre_2', 'Genre_3'],  # แปลงคอลัมน์ Genre_1, Genre_2, Genre_3 ให้อยู่ในแนวตั้ง
        var_name='Genre_Type',
        value_name='Genre'
    )
    .drop_duplicates(subset=['year', 'show_title', 'Genre'])  # ลบ Genre ที่ซ้ำกันในรายการเดียวกัน
)

# นับจำนวนครั้งที่แต่ละ Genre ปรากฏในแต่ละปี
genre_counts = df_melted.groupby(['year', 'Genre']).size().reset_index(name='count')

# พล็อต Heatmap เพื่อแสดงการกระจายของประเภทหนังในแต่ละปี
heatmap = alt.Chart(genre_counts).mark_rect().encode(
    x=alt.X('year:O', title="Year"),
    y=alt.Y('Genre:N', title="Genre", sort='-x'),
    color=alt.Color(
        'count:Q',
        scale=alt.Scale(scheme='reds'),  # ใช้โทนสีแดงให้เห็นความเข้มของจำนวนที่มากขึ้น
        legend=alt.Legend(title="Count")
    ),
    tooltip=['year', 'Genre', 'count']
).properties(
    width=600,
    height=400,
)

# แสดงผลลัพธ์
heatmap


alt.Chart(...)

#วิเคราะห์ว่าช่วงเวลาไหนที่ซีรีส์มียอดวิวสูงที่สุด

In [114]:


# แปลงคอลัมน์ 'week' เป็น datetime
df_most_hour['week'] = pd.to_datetime(df_most_hour['week'], format='mixed')

# สร้างคอลัมน์ 'year' และ 'month'
df_most_hour['year'] = df_most_hour['week'].dt.year
df_most_hour['month'] = df_most_hour['week'].dt.month

# แปลงตัวเลขเดือนเป็นชื่อเดือนย่อ (Jan, Feb, ...)
df_most_hour['month_abbr'] = df_most_hour['month'].apply(lambda x: calendar.month_abbr[x])

# คำนวณยอดวิวรายเดือนสำหรับแต่ละปี
df_monthly_views = df_most_hour.groupby(['year', 'month_abbr'])['weekly_hours_viewed'].sum().reset_index()

# สร้างลำดับเดือนให้เรียงตาม Jan → Dec
month_order = list(calendar.month_abbr[1:])  # ['Jan', 'Feb', 'Mar', ..., 'Dec']

# สร้างกราฟแนวโน้มยอดวิวรายเดือนของแต่ละปี
line_chart = alt.Chart(df_monthly_views).mark_line(point=True).encode(
    x=alt.X('month_abbr:N', title='Month', axis=alt.Axis(labelAngle=0), sort=month_order),  # กำหนดการเรียงเดือน
    y=alt.Y('weekly_hours_viewed:Q', title='Total Monthly Hours Viewed'),
    color=alt.Color('year:N', title='Year', legend=alt.Legend(title="Year")),
    tooltip=['year', 'month_abbr', 'weekly_hours_viewed']
).properties(
    width=800,
    height=500
)

# ค้นหาช่วงเวลาที่ยอดวิวสูงสุดในแต่ละปี
peak_months = df_monthly_views.loc[df_monthly_views.groupby('year')['weekly_hours_viewed'].idxmax()]

# ค้นหาซีรีส์ที่มียอดวิวสูงสุดในช่วงพีคของแต่ละปี
top_shows = []
for _, row in peak_months.iterrows():
    top_show = df_most_hour[
        (df_most_hour['year'] == row['year']) & (df_most_hour['month_abbr'] == row['month_abbr'])
    ].nlargest(1, 'weekly_hours_viewed')
    top_shows.append(top_show)

# รวมข้อมูลซีรีส์ยอดนิยมที่ขับเคลื่อนยอดวิวสูงสุดในแต่ละปี
df_top_shows = pd.concat(top_shows)

# แสดงข้อมูลซีรีส์ที่ทำให้ยอดวิวสูงสุดในแต่ละปี
display(df_top_shows[['year', 'month_abbr', 'show_title', 'weekly_hours_viewed']])

# แสดงกราฟ
line_chart


,year,month_abbr,show_title,weekly_hours_viewed
6750,2021,Oct,Squid Game,571760000
5180,2022,Jul,Stranger Things,301280000
4100,2023,Jan,Ginny & Georgia,180470000
500,2024,Sep,Monsters,153800000


alt.Chart(...)

วิเคราะห์ความสัมพันธ์ระหว่าง runtime และ ยอดวิว (weekly hours viewed) สำหรับซีรีส์ ทั้ง English และ Non-English เพื่อดูว่ามีความสัมพันธ์กันหรือไม่ โดยใช้ การวิเคราะห์สหสัมพันธ์ (correlation analysis) และ scatter plot เพื่อตรวจสอบแนวโน้มของข้อมูล

In [115]:
df_filtered = df_most_hour.copy()  # คัดลอก DataFrame
df_filtered = df_filtered[df_filtered['runtime'] > 0]

# เลือกเฉพาะคอลัมน์ที่ต้องใช้ และลบค่าที่หายไป
df_filtered = df_filtered[['runtime', 'weekly_hours_viewed']].dropna()

# สร้าง Scatter Plot
chart = alt.Chart(df_filtered).mark_circle(size=60, opacity=0.5).encode(
    x=alt.X('runtime', title='Runtime (hours)'),
    y=alt.Y('weekly_hours_viewed', title='Weekly Hours Viewed (millions)'),
    tooltip=['runtime', 'weekly_hours_viewed']
).properties(
    width=600,
    height=400
).interactive()

# แสดงกราฟ
chart


alt.Chart(...)